# Model Explainability Analysis - SHAP

This notebook analyzes the champion model using SHAP (SHapley Additive exPlanations):
- Global feature importance
- Local explanations for individual predictions
- Business insights from model decisions

## 1. Setup and Imports

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import shap
import joblib
import mlflow

# Import custom modules
from explainability import *
from metrics import *

print("All imports successful!")
print(f"SHAP version: {shap.__version__}")

## 2. Load Champion Model

In [ ]:
# Load the comparison results to identify champion
comparison_df = pd.read_csv('../reports/model_comparison.csv')
best_model_idx = comparison_df['Business Cost'].idxmin()
best_model_name = comparison_df.loc[best_model_idx, 'Model']

print(f"Champion Model: {best_model_name}")
print(f"Business Cost: {comparison_df.loc[best_model_idx, 'Business Cost']:.2f}")
print(f"AUC: {comparison_df.loc[best_model_idx, 'AUC']:.4f}")

# Map model name to file
model_files = {
    'Logistic Regression': 'logistic_regression.pkl',
    'Random Forest': 'random_forest.pkl',
    'XGBoost': 'xgboost.pkl',
    'LightGBM': 'lightgbm.pkl'
}

model_file = model_files[best_model_name]
model = joblib.load(f'../models/{model_file}')
print(f"\nModel loaded from ../models/{model_file}")

## 3. Load Test Data

In [ ]:
# Load prepared data
df = pd.read_csv('../data/application_train_prepared.csv')

# Separate features and target
if 'TARGET' in df.columns:
    target_col = 'TARGET'
else:
    target_col = [col for col in df.columns if 'target' in col.lower()][0]

X = df.drop(columns=[target_col])
y = df[target_col]

# Use same split as training notebook
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Test set size: {X_test.shape[0]:,}")
print(f"Number of features: {X_test.shape[1]:,}")

## 4. Sample Data for SHAP (Performance)

In [ ]:
# Sample for performance (SHAP can be slow on large datasets)
sample_size = min(1000, X_test.shape[0])
X_test_sample = X_test.sample(n=sample_size, random_state=42)
y_test_sample = y_test.loc[X_test_sample.index]

print(f"Using {sample_size} samples for SHAP analysis")

## 5. Create SHAP Explainer

In [ ]:
# Create appropriate explainer based on model type
if best_model_name == 'Logistic Regression':
    # For linear models
    explainer = shap.LinearExplainer(model, X_train)
else:
    # For tree-based models (RF, XGB, LGB)
    explainer = shap.TreeExplainer(model)

print(f"SHAP Explainer created for {best_model_name}")

## 6. Calculate SHAP Values

In [ ]:
# Calculate SHAP values
print("Calculating SHAP values... (this may take a few minutes)")
shap_values = explainer.shap_values(X_test_sample)

# For binary classification, some explainers return values for both classes
if isinstance(shap_values, list):
    shap_values = shap_values[1]  # Use positive class

print(f"SHAP values shape: {shap_values.shape}")
print("SHAP calculation complete!")

## 7. Global Feature Importance - Summary Plot

In [ ]:
# SHAP Summary Plot (beeswarm)
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test_sample, show=False, max_display=20)
plt.tight_layout()
os.makedirs('../reports/figures', exist_ok=True)
plt.savefig('../reports/figures/shap_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print("SHAP summary plot saved to ../reports/figures/shap_summary.png")

## 8. Global Feature Importance - Bar Plot

In [ ]:
# SHAP Bar Plot (mean absolute SHAP values)
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test_sample, plot_type='bar', show=False, max_display=20)
plt.tight_layout()
plt.savefig('../reports/figures/shap_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("SHAP importance plot saved to ../reports/figures/shap_importance.png")

## 9. Top 20 Most Important Features

In [ ]:
# Calculate mean absolute SHAP values
mean_abs_shap = np.abs(shap_values).mean(axis=0)
feature_importance = pd.DataFrame({
    'feature': X_test_sample.columns,
    'importance': mean_abs_shap
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features (by mean |SHAP value|):")
print("="*60)
print(feature_importance.head(20).to_string(index=False))
print("="*60)

## 10. Feature Impact Analysis

In [ ]:
# Analyze which features increase vs decrease default risk
print("\nFeature Impact Analysis:")
print("="*60)

top_features = feature_importance.head(10)['feature'].tolist()

for feature in top_features:
    feature_idx = X_test_sample.columns.get_loc(feature)
    mean_shap = shap_values[:, feature_idx].mean()
    
    if mean_shap > 0:
        direction = "INCREASES default risk"
    else:
        direction = "DECREASES default risk"
    
    print(f"{feature:40s} -> {direction} (mean SHAP: {mean_shap:+.4f})")

print("="*60)

## 11. Business Interpretation

### Key Insights:

Based on the SHAP analysis above:

1. **External Data Sources** (EXT_SOURCE_X): Often the most predictive features
2. **Credit Amount**: Higher loan amounts may correlate with risk
3. **Age/Employment**: Client stability indicators
4. **Previous Credit History**: Strong predictor of future behavior

These insights should guide:
- Which data to prioritize collecting
- What to monitor in production
- How to explain decisions to stakeholders

## 12. Select Interesting Clients for Local Analysis

In [ ]:
# Get predictions
y_pred_proba = model.predict_proba(X_test_sample)[:, 1]
y_pred = (y_pred_proba >= 0.5).astype(int)

# Find interesting cases
true_positives = X_test_sample[(y_test_sample == 1) & (y_pred == 1)].index
true_negatives = X_test_sample[(y_test_sample == 0) & (y_pred == 0)].index
false_positives = X_test_sample[(y_test_sample == 0) & (y_pred == 1)].index
false_negatives = X_test_sample[(y_test_sample == 1) & (y_pred == 0)].index

# Edge cases (probability near 0.5)
edge_cases = X_test_sample[(y_pred_proba > 0.45) & (y_pred_proba < 0.55)].index

# Select one of each
interesting_clients = []
interesting_labels = []

if len(true_positives) > 0:
    interesting_clients.append(true_positives[0])
    interesting_labels.append('True Positive (Correct Default Prediction)')

if len(true_negatives) > 0:
    interesting_clients.append(true_negatives[0])
    interesting_labels.append('True Negative (Correct Non-Default Prediction)')

if len(false_positives) > 0:
    interesting_clients.append(false_positives[0])
    interesting_labels.append('False Positive (Conservative Error)')

if len(false_negatives) > 0:
    interesting_clients.append(false_negatives[0])
    interesting_labels.append('False Negative (Costly Error)')

if len(edge_cases) > 0:
    interesting_clients.append(edge_cases[0])
    interesting_labels.append('Edge Case (Probability ~0.5)')

print(f"Selected {len(interesting_clients)} interesting clients for local analysis")

## 13. Local Explanations - Waterfall Plots

In [ ]:
# Create waterfall plot for each interesting client
for idx, (client_idx, label) in enumerate(zip(interesting_clients, interesting_labels)):
    print(f"\n{'='*80}")
    print(f"Client Analysis: {label}")
    print(f"{'='*80}")
    
    # Get client position in sample
    client_pos = X_test_sample.index.get_loc(client_idx)
    
    # Client info
    actual = y_test_sample.loc[client_idx]
    predicted_proba = y_pred_proba[client_pos]
    predicted = y_pred[client_pos]
    
    print(f"Actual: {actual} | Predicted: {predicted} | Probability: {predicted_proba:.4f}")
    
    # Waterfall plot
    plt.figure(figsize=(10, 8))
    shap.waterfall_plot(
        shap.Explanation(
            values=shap_values[client_pos],
            base_values=explainer.expected_value if hasattr(explainer, 'expected_value') else 0,
            data=X_test_sample.iloc[client_pos],
            feature_names=X_test_sample.columns.tolist()
        ),
        max_display=15,
        show=False
    )
    plt.title(f"{label}\nActual: {actual}, Predicted: {predicted}, Prob: {predicted_proba:.3f}")
    plt.tight_layout()
    plt.savefig(f'../reports/figures/shap_waterfall_client_{idx+1}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Top contributing features
    client_shap = shap_values[client_pos]
    top_features_idx = np.argsort(np.abs(client_shap))[-10:][::-1]
    
    print(f"\nTop 10 Contributing Features:")
    for feat_idx in top_features_idx:
        feat_name = X_test_sample.columns[feat_idx]
        feat_value = X_test_sample.iloc[client_pos, feat_idx]
        shap_val = client_shap[feat_idx]
        print(f"  {feat_name:35s} = {feat_value:10.2f}  (SHAP: {shap_val:+.4f})")

## 14. Create Complete Explainability Report

In [ ]:
# Use the custom explainability function
print("Generating complete explainability report...")

report = create_explainability_report(
    model=model,
    X_test=X_test_sample,
    y_test=y_test_sample,
    feature_names=X_test_sample.columns.tolist(),
    output_dir='../reports/figures',
    model_name=best_model_name
)

print("\nExplainability report generated!")
print("All visualizations saved to ../reports/figures/")

## 15. Export Critical Features List

In [ ]:
# Save top features to CSV
feature_importance.head(50).to_csv('../reports/critical_features.csv', index=False)
print("Critical features saved to ../reports/critical_features.csv")

## 16. Business Recommendations

### Key Business Recommendations:

#### 1. Data Collection Priorities
Based on feature importance, prioritize collecting:
- External credit scores (EXT_SOURCE variables)
- Complete employment history
- Previous credit behavior patterns
- Income and asset information

#### 2. Red Flag Signals
Indicators that strongly predict default:
- Low external credit scores
- High credit amount relative to income
- Short employment duration
- Previous payment difficulties

#### 3. Risk Reduction Factors
Factors that reduce default probability:
- Higher external credit scores
- Stable employment history
- Lower debt-to-income ratio
- Good previous credit history

#### 4. Model Improvement Suggestions
- Add interaction features between key variables
- Collect more granular employment data
- Monitor feature drift in production
- A/B test different threshold strategies
- Consider ensemble methods combining multiple models

## Summary

This notebook has:
1. ✅ Loaded the champion model
2. ✅ Calculated SHAP values for global interpretability
3. ✅ Generated summary and importance plots
4. ✅ Analyzed local explanations for interesting cases
5. ✅ Provided business insights and recommendations

### Next Steps:
- Review all generated figures in `../reports/figures/`
- Proceed to `04_mlflow_serving_test.ipynb` for deployment testing